In [ ]:
# Импорт необходимых библиотек
import gc
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import  SGD, RMSprop, Adam
from functools import cmp_to_key
from google.colab import drive

drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new')

# Определение графа с рёбрами, начальной и конечной вершинами
edges = [[1, 2], [1, 5], [1, 8], [2, 3], [3, 4], [3, 8], [3, 9], [4, 10], [5, 6], [6, 7], [6, 8], [6, 9], [7, 10], [9, 10]]
number_edges = len(edges)  # Количество рёбер
number_steps = 4  # Количество шагов в последовательности
start_node = 1  # Начальная вершина
end_node = 10  # Конечная вершина

# Загрузка данных из CSV файлов
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU.csv')
x = df.iloc[:,:number_edges].values  # Признаки
y = df.iloc[:,number_edges:].values  # Метки

df_hp = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU_hp.csv')
x_hp = df_hp.iloc[:,:number_edges].values  # Признаки для гиперпараметров
y_hp = df_hp.iloc[:,number_edges:].values  # Метки для гиперпараметров

# Разделение данных на обучающий, тестовый и валидационный наборы
from sklearn.model_selection import train_test_split
x_main_train, x_main_tv, y_main_train, y_main_tv = train_test_split(x, y, test_size=2/5)
x_main_test, x_main_valid, y_main_test, y_main_valid = train_test_split(x_main_tv, y_main_tv, test_size=1/2)

x_hp_train, x_hp_tv, y_hp_train, y_hp_tv = train_test_split(x_hp, y_hp, test_size=2/5)
x_hp_test, x_hp_valid, y_hp_test, y_hp_valid = train_test_split(x_hp_tv, y_hp_tv, test_size=1/2)

# Стандартизация данных
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_main_train)
x_test = sc.transform(x_main_test)
x_valid = sc.transform(x_main_valid)

sc_hp = StandardScaler()
x_train_1 = sc_hp.fit_transform(x_hp_train)
x_test_1 = sc_hp.transform(x_hp_test)
x_valid_1 = sc_hp.transform(x_hp_valid)

# Расширение размерности данных для временных рядов
x_train = np.repeat(x_train[:, np.newaxis, : ], number_steps, axis=1)
x_test = np.repeat(x_test[:, np.newaxis, : ], number_steps, axis=1)
x_valid = np.repeat(x_valid[:, np.newaxis, : ], number_steps, axis=1)

x_train_1 = np.repeat(x_train_1[:, np.newaxis, : ], number_steps, axis=1)
x_test_1 = np.repeat(x_test_1[:, np.newaxis, : ], number_steps, axis=1)
x_valid_1 = np.repeat(x_valid_1[:, np.newaxis, : ], number_steps, axis=1)

# Изменение формы меток для соответствия размерности
y_train = np.reshape(y_main_train, (y_main_train.shape[0], number_steps, number_edges))
y_test = np.reshape(y_main_test, (y_main_test.shape[0], number_steps, number_edges))
y_valid = np.reshape(y_main_valid, (y_main_valid.shape[0], number_steps, number_edges))

y_train_1 = np.reshape(y_hp_train, (y_hp_train.shape[0], number_steps, number_edges))
y_test_1 = np.reshape(y_hp_test, (y_hp_test.shape[0], number_steps, number_edges))
y_valid_1 = np.reshape(y_hp_valid, (y_hp_valid.shape[0], number_steps, number_edges))

# Вывод формы данных
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape, x_train_1.shape, y_train_1.shape, x_test_1.shape, y_test_1.shape)

# Определение гиперпараметров структуры и обучения
# Гиперпараметры структуры: количество слоёв, количество единиц, dropout, тип ячейки
# Гиперпараметры обучения: скорость обучения, размер батча
HP_1 = [[1, 2, 3, 4, 5],
       [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, -1],
       [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
       [SimpleRNN, GRU, LSTM],
       [SGD, RMSprop, Adam],
       [0.0001, 0.001, 0.01, 0.1],
       [16, 32, 64, 128, 256, 512, 1024]]

# Создание списка количества вариантов гиперпараметров
t = []
for p in HP_1:
    t.append(len(p))

# Инициализация вспомогательных переменных
t1 = []
hp = []
for i in range(t[0]):
    t1.append(t[1])
    t1.append(t[2])
    hp.append(HP_1[1])
    hp.append(HP_1[2])

# Добавление гиперпараметров, начиная с третьего элемента
for i in range(3, len(HP_1)):
    t1.append(t[i])
    hp.append(HP_1[i])

# Определение квадратных корней количества вариантов гиперпараметров
t2 = [int(np.sqrt(itm)) for itm in t1]

# Расчёт общего количества гиперпараметров
n_hp = 2*t[0] + len(HP_1) - 3

# Вывод информации о гиперпараметрах
print(t, t1, t2, n_hp)

# Определение пользовательской функции точности
def custom_accuracy(y_true, y_pred):
    threshold = 0.5
    y_pred_binary = tf.cast(tf.greater_equal(y_pred, threshold), tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    stepwise_correct = tf.reduce_all(tf.equal(y_true, y_pred_binary), axis=-1)
    sequencewise_correct = tf.reduce_all(stepwise_correct, axis=-1)
    acc = tf.reduce_mean(tf.cast(sequencewise_correct, tf.float32))
    return acc

# Создание callback для ранней остановки при обучении модели
callback = tf.keras.callbacks.EarlyStopping(monitor='val_custom_accuracy',
                                            min_delta=0.001, patience=10, verbose=1, mode='max',
                                            restore_best_weights=True)

Mounted at /content/gdrive


In [ ]:
class Network:
    def __init__(self):
        # Инициализация сети: генерируем решение, инициализируем значения фитнеса, счётчика и вероятности
        self.gen = self.create_solution()
        self.fitness = 0
        self.counter = 0
        self.prob = 0
        self.velocity = self.create_velocity()  # Создаём начальную скорость

    def create_velocity(self):
        # Создание начальной скорости для каждого параметра
        v = []
        for i in range(n_hp):
            v.append(np.random.randint(-t2[i], t2[i] + 1))  # Скорость генерируется случайным образом в пределах [-t2, t2]
        return v
    
    def create_solution(self):
        # Создание решения (генерация набора гиперпараметров)
        gen = []
        for i in range(n_hp):
            gen.append(np.random.choice(list(range(t1[i]))))  # Выбираем случайный гиперпараметр для каждого элемента
        return gen

    def check_gen(self):
        # Проверка генов (гиперпараметров), чтобы убедиться, что они допустимы
        state = False
        for i in range(n_hp):
            # Если ген относится к числу основных гиперпараметров и не равен -1
            if i % 2 == 0 and i < 2 * t[0] and hp[i][self.gen[i]] != -1:
                state = True  # Разрешено использовать этот набор гиперпараметров
                break
        return state

    def evaluate(self):
        # Оценка модели на основе текущего решения (набора гиперпараметров)
        if self.check_gen() == False:
            # Если гиперпараметры недопустимы, возвращаем нулевую оценку
            return 0
        else:
            # Определяем входные данные модели (временные ряды с шагами и рёбрами)
            input_shape = (number_steps, number_edges)
            inputs = Input(shape=input_shape)  # Входной слой
            x_input = inputs
            cell_type = hp[2*t[0]][self.gen[2*t[0]]]  # Определяем тип ячейки (SimpleRNN, GRU или LSTM)

            # Построение рекуррентных слоёв на основе гиперпараметров
            for i in range(t[0]):
                if hp[2*i][self.gen[2*i]] != -1:
                    x_input = cell_type(hp[2*i][self.gen[2*i]], return_sequences=True)(x_input)  # Добавление рекуррентного слоя
                    x_input = Dropout(hp[2*i+1][self.gen[2*i+1]])(x_input)  # Dropout слой для предотвращения переобучения

            # Выходной слой (TimeDistributed) с сигмоидной активацией для прогнозирования по каждому шагу последовательности
            outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
            model = Model(inputs=inputs, outputs=outputs)  # Определение модели

            # Компиляция модели с выбранным оптимизатором и скоростью обучения
            opt = hp[2*t[0]+1][self.gen[2*t[0]+1]]
            model.compile(loss=BinaryCrossentropy(), 
                          optimizer=opt(learning_rate=hp[2*t[0]+2][self.gen[2*t[0]+2]]), 
                          metrics=[custom_accuracy])

            # Обучение модели на обучающем наборе данных с использованием callback для ранней остановки
            model.fit(x_train_1, y_train_1, 
                      epochs=100, 
                      callbacks=[callback], 
                      validation_data=(x_valid_1, y_valid_1), 
                      batch_size=hp[2*t[0]+3][self.gen[2*t[0]+3]], 
                      verbose=0)

            # Прогнозирование и вычисление точности на тестовом наборе данных
            pred = model.predict(x_test_1)
            accuracy_hp = custom_accuracy(y_test_1, pred)
            accuracy_result = tf.keras.backend.get_value(accuracy_hp)
            acc = max(int(round(accuracy_result * 100)), 1)  # Округляем точность и гарантируем, что значение не меньше 1

            # Очистка сессии Keras для предотвращения утечек памяти
            tf.keras.backend.clear_session()
            return acc  # Возвращаем оценку точности

In [ ]:
class ABC:
    def __init__(self, N, Max):
        # Инициализация алгоритма ABC (Artificial Bee Colony)
        self.N = N  # Размер популяции
        self.Max = Max  # Максимальное количество итераций
        self.population = []  # Популяция агентов
        self.best = Network()  # Лучшая сеть
        self.limit = self.Max / 2  # Лимит для фазы разведчика
        self.pool = []  # Пул для хранения уже проверенных решений

    def population_fitness(self):
        # Вычисление фитнеса для всей популяции
        for i in range(len(self.population)):
            self.calculate_fitness(self.population[i])
            self.show(self.population[i])
        print("__________________________________________________________________")

    def calculate_fitness(self, new_mem):
        # Вычисление фитнеса для нового решения
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):  # Проверка, было ли уже это решение
                cond = False
                new_mem.fitness = mem.fitness  # Используем сохранённый фитнес
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()  # Вычисляем фитнес для нового решения
            self.pool.append(copy.deepcopy(new_mem))  # Сохраняем решение в пул

    def complexity(self, m):
        # Вычисление сложности модели
        coeff = None
        r_a = 0
        if m.gen[2 * t[0]] == 0:
            coeff = 1  # Коэффициент для SimpleRNN
        elif m.gen[2 * t[0]] == 1:
            coeff = 3  # Коэффициент для GRU
            r_a = 1
        else:
            coeff = 4  # Коэффициент для LSTM
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2 * i][m.gen[2 * i]] != -1:
                b = coeff * hp[2 * i][m.gen[2 * i]] * (hp[2 * i][m.gen[2 * i]] + g + r_a + 1)
                a += b
                g = hp[2 * i][m.gen[2 * i]]
        a += number_edges * (g + 1)
        return a

    def compare(self, m1, m2):
        # Сравнение двух решений по фитнесу и сложности
        if m1.fitness == m2.fitness:
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                return 0
            else:
                return 1
        elif m1.fitness > m2.fitness:
            return -1
        else:
            return 1

    def initialization_phase(self):
        # Инициализация начальной популяции
        self.population = [Network() for i in range(self.N)]

    def employed_phase(self):
        # Фаза рабочих пчёл
        print("employed_phase:")
        population = copy.deepcopy(self.population)
        for i in range(self.N):
            r = list(range(0, i)) + list(range(i + 1, self.N))
            coceg = np.random.choice(r)  # Выбираем случайную особь
            d = np.random.randint(n_hp)  # Выбираем случайный гиперпараметр
            fi = np.random.uniform(-1, 1)  # Генерируем случайный коэффициент
            solution = copy.deepcopy(self.population[i])
            ax = round(solution.gen[d] + fi * (solution.gen[d] - self.population[coceg].gen[d]))
            solution.gen[d] = max(0, min(ax, t1[d] - 1))  # Обновляем значение гиперпараметра
            self.calculate_fitness(solution)
            if self.compare(solution, population[i]) == -1:
                print("Change at: i = {}, d = {}".format(i, d))
                solution.counter = 0
                population[i] = copy.deepcopy(solution)
                self.show(population[i])
                print("__________________________________________________________________")
            else:
                population[i].counter += 1
        self.population = copy.deepcopy(population)

    def onlooker_phase(self):
        # Фаза пчёл-наблюдателей
        print("onlooker_phase:")
        summ = 0
        for i in range(self.N):
            summ += self.population[i].fitness
        for i in range(self.N):
            self.population[i].prob = self.population[i].fitness / summ  # Вычисляем вероятность выбора
        prob = [a.prob for a in self.population]
        population = copy.deepcopy(self.population)
        for i in range(self.N):
            index_solution = np.random.choice(list(range(self.N)), p=prob)  # Выбираем решение пропорционально его вероятности
            r = list(range(index_solution)) + list(range(index_solution + 1, self.N))
            coceg = np.random.choice(r)
            d = np.random.randint(n_hp)
            fi = np.random.uniform(-1, 1)
            solution = copy.deepcopy(self.population[index_solution])
            ax = round(solution.gen[d] + fi * (solution.gen[d] - self.population[coceg].gen[d]))
            solution.gen[d] = max(0, min(ax, t1[d] - 1))
            self.calculate_fitness(solution)
            if self.compare(solution, population[index_solution]) == -1:
                print("Change at: i = {}, d = {}".format(index_solution, d))
                solution.counter = 0
                population[index_solution] = copy.deepcopy(solution)
                self.show(population[index_solution])
                print("__________________________________________________________________")
            else:
                population[index_solution].counter += 1
        self.population = copy.deepcopy(population)

    def scout_phase(self):
        # Фаза пчёл-разведчиков (замена неудачных решений)
        print("scout_phase:")
        for i in range(self.N):
            if self.population[i].counter >= self.limit:
                print("Change at: i = ", i)
                self.population[i] = Network()  # Генерируем новое решение
                self.calculate_fitness(self.population[i])

    def current_population(self):
        # Отображение текущей популяции
        for i in range(len(self.population)):
            self.show(self.population[i])
        print("__________________________________________________________________")

    def get_best(self):
        # Нахождение лучшего решения
        self.population.sort(key=cmp_to_key(self.compare))  # Сортируем популяцию по фитнесу
        self.current_population()
        if self.compare(self.best, self.population[0]) == 1:
            self.best = copy.deepcopy(self.population[0])  # Обновляем лучшее решение
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        # Отображение информации о решении
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2 * j], m.gen[2 * j + 1]))
        p_copy.append(m.gen[2 * t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        # Основной цикл выполнения алгоритма
        self.initialization_phase()
        self.population_fitness()
        for itr in range(self.Max):
            print("iteration = ", itr)
            self.employed_phase()  # Фаза рабочих пчёл
            self.current_population()
            self.onlooker_phase()  # Фаза наблюдателей
            self.current_population()
            self.scout_phase()  # Фаза разведчиков
            self.current_population()
            self.get_best()  # Получаем лучшее решение
            gc.collect()  # Очистка памяти

In [ ]:
class BFA:
    def __init__(self, N, Max, w, c1, c2):
        # Инициализация алгоритма стаи птиц
        self.N = N  # Количество птиц
        self.Max = Max  # Максимальное количество итераций
        self.w = w  # Инерционный коэффициент
        self.c1 = c1  # Когнитивный коэффициент
        self.c2 = c2  # Социальный коэффициент
        self.best_global_solution = Network()  # Лучшая глобальная сеть
        self.population = [Network() for i in range(self.N)]  # Инициализация популяции птиц
        self.best_local_solution = [copy.deepcopy(self.population[i]) for i in range(self.N)]  # Локальные решения
        self.best = Network()  # Лучшая найденная сеть
        self.pool = []  # Пул решений

    def population_fitness(self):
        # Вычисление фитнеса для всей популяции птиц
        for i in range(len(self.population)):
            self.calculate_fitness(self.population[i])
            self.show(self.population[i])
        print("__________________________________________________________________")

    def calculate_fitness(self, new_mem):
        # Вычисление фитнеса для нового решения
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):
                cond = False
                new_mem.fitness = mem.fitness
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()
            self.pool.append(copy.deepcopy(new_mem))

    def complexity(self, m):
        # Вычисление сложности модели
        coeff = None
        r_a = 0
        if m.gen[2 * t[0]] == 0:
            coeff = 1  # SimpleRNN
        elif m.gen[2 * t[0]] == 1:
            coeff = 3  # GRU
            r_a = 1
        else:
            coeff = 4  # LSTM
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2 * i][m.gen[2 * i]] != -1:
                b = coeff * hp[2 * i][m.gen[2 * i]] * (hp[2 * i][m.gen[2 * i]] + g + r_a + 1)
                a += b
                g = hp[2 * i][m.gen[2 * i]]
        a += number_edges * (g + 1)
        return a

    def compare(self, m1, m2):
        # Сравнение двух решений на основе их фитнеса и сложности
        if m1.fitness == m2.fitness:
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                return 0
            else:
                return 1
        elif m1.fitness > m2.fitness:
            return -1
        else:
            return 1

    def update_local_global(self, solution, best_local_solution):
        # Обновление локального и глобального решения
        if self.compare(solution, best_local_solution) == -1:
            best_local_solution = copy.deepcopy(solution)  # Обновляем локальное решение
            if self.compare(best_local_solution, self.best_global_solution) == -1:
                self.best_global_solution = copy.deepcopy(best_local_solution)  # Обновляем глобальное решение

    def update_velocity_position(self):
        # Обновление скорости и позиции для каждой птицы
        for i in range(self.N):
            velocity = copy.deepcopy(self.population[i].velocity)  # Сохраняем текущую скорость
            gen = copy.deepcopy(self.population[i].gen)  # Сохраняем текущую позицию

            # Вычисляем новую скорость
            for j in range(n_hp):
                r1 = np.random.rand()
                r2 = np.random.rand()
                v1 = self.c1 * r1 * (self.best_local_solution[i].gen[j] - self.population[i].gen[j])
                v2 = self.c2 * r2 * (self.best_global_solution.gen[j] - self.population[i].gen[j])
                velocity[j] = round(self.w * self.population[i].velocity[j] + v1 + v2)
                velocity[j] = max(-t2[j], min(velocity[j], t2[j]))  # Ограничиваем скорость в пределах [-t2, t2]

            # Обновляем позицию птиц
            for j in range(n_hp):
                a = round(self.population[i].gen[j] + velocity[j])
                gen[j] = max(0, min(a, t1[j] - 1))  # Ограничиваем позицию в допустимых пределах

            self.population[i].velocity = copy.deepcopy(velocity)  # Обновляем скорость птицы
            self.population[i].gen = copy.deepcopy(gen)  # Обновляем позицию птицы
            self.calculate_fitness(self.population[i])  # Вычисляем фитнес
            self.show(self.population[i])
            print("__________________________________________________________________")
            self.update_local_global(self.population[i], self.best_local_solution[i])  # Обновляем локальное и глобальное решение

    def current_population(self):
        # Отображение текущей популяции птиц
        for i in range(len(self.population)):
            self.show(self.population[i])
        print("__________________________________________________________________")

    def get_best(self):
        # Получение лучшего решения
        self.population.sort(key=cmp_to_key(self.compare))
        self.current_population()
        if self.compare(self.best, self.population[0]) == 1:
            self.best = copy.deepcopy(self.population[0])  # Обновляем лучшее решение
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        # Отображение информации о решении
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2 * j], m.gen[2 * j + 1]))
        p_copy.append(m.gen[2 * t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        # Основной цикл выполнения алгоритма BFA
        for itr in range(self.Max):
            print("iteration = ", itr)
            self.update_velocity_position()  # Обновление скорости и позиции птиц
            self.current_population()  # Отображение текущей популяции птиц
            self.get_best()  # Получение лучшего решения
            gc.collect()  # Очистка памяти

In [ ]:
class FA:
    def __init__(self, N, Max, y, a0, b0):
        # Инициализация алгоритма светлячков (FA)
        self.N = N  # Количество светлячков
        self.Max = Max  # Максимальное количество итераций
        self.population = [Network() for i in range(self.N)]  # Инициализация популяции
        self.best = Network()  # Лучшая сеть
        self.pool = []  # Пул уже проверенных решений
        self.y = y  # Коэффициент поглощения света
        self.a0 = a0  # Параметр, контролирующий случайные шаги
        self.b0 = b0  # Базовый коэффициент яркости светлячков
        self.a = 0  # Текущий параметр, контролирующий случайные шаги

    def population_fitness(self):
        # Вычисление фитнеса для всей популяции светлячков
        for i in range(len(self.population)):
            self.calculate_fitness(self.population[i])
            self.show(self.population[i])
        print("__________________________________________________________")

    def calculate_fitness(self, new_mem):
        # Вычисление фитнеса для нового решения
        cond = True
        for mem in self.pool:
            if tuple(new_mem.gen) == tuple(mem.gen):
                cond = False
                new_mem.fitness = mem.fitness
                break
        if cond == True:
            new_mem.fitness = new_mem.evaluate()
            self.pool.append(copy.deepcopy(new_mem))

    def complexity(self, m):
        # Вычисление сложности модели
        coeff = None
        r_a = 0
        if m.gen[2 * t[0]] == 0:
            coeff = 1  # SimpleRNN
        elif m.gen[2 * t[0]] == 1:
            coeff = 3  # GRU
            r_a = 1
        else:
            coeff = 4  # LSTM
        a = 0
        g = number_edges
        for i in range(t[0]):
            if hp[2 * i][m.gen[2 * i]] != -1:
                b = coeff * hp[2 * i][m.gen[2 * i]] * (hp[2 * i][m.gen[2 * i]] + g + r_a + 1)
                a += b
                g = hp[2 * i][m.gen[2 * i]]
        a += number_edges * (g + 1)
        return a

    def compare(self, m1, m2):
        # Сравнение двух решений на основе их фитнеса и сложности
        if m1.fitness == m2.fitness:
            a = self.complexity(m1)
            b = self.complexity(m2)
            if a < b:
                return -1
            elif a == b:
                return 0
            else:
                return 1
        elif m1.fitness > m2.fitness:
            return -1
        else:
            return 1

    def attract(self):
        # Притяжение светлячков с учётом их фитнеса
        for i in range(self.N):
            for j in range(self.N):
                newSolution = Network()
                if self.compare(self.population[i], self.population[j]) == 1:
                    # Рассчитываем расстояние между светлячками
                    r2 = 0
                    for j1 in range(n_hp):
                        number = (self.population[i].gen[j1] - self.population[j].gen[j1]) / t1[j1]
                        r2 += pow(number, 2)
                    r2 = r2 / n_hp

                    # Рассчитываем коэффициент притяжения
                    b = self.b0 * math.exp(-self.y * r2)

                    # Обновляем позицию решения
                    for j1 in range(n_hp):
                        e = random.uniform(0, 1) - 1 / 2  # Случайная компонента
                        ax = round(self.population[i].gen[j1] + b * (self.population[j].gen[j1] - self.population[i].gen[j1]) + t1[j1] * self.a * e)
                        newSolution.gen[j1] = max(0, min(ax, t1[j1] - 1))  # Ограничиваем обновленное решение в допустимых пределах

                    self.calculate_fitness(newSolution)  # Вычисляем фитнес обновлённого решения
                    self.population[i] = copy.deepcopy(newSolution)  # Обновляем популяцию
                    self.show(self.population[i])
                    print("__________________________________________________________________")

    def current_population(self):
        # Отображение текущей популяции светлячков
        for i in range(len(self.population)):
            self.show(self.population[i])
        print("__________________________________________________________________")

    def get_best(self):
        # Получение лучшего решения
        self.population.sort(key=cmp_to_key(self.compare))
        self.current_population()
        if self.compare(self.best, self.population[0]) == 1:
            self.best = copy.deepcopy(self.population[0])  # Обновляем лучшее решение
        self.show(self.best)
        print("__________________________________________________________________")

    def show(self, m):
        # Отображение информации о решении
        p_copy = []
        for j in range(t[0]):
            p_copy.append((m.gen[2 * j], m.gen[2 * j + 1]))
        p_copy.append(m.gen[2 * t[0]:])
        print(p_copy, m.fitness)

    def do(self):
        # Основной цикл выполнения алгоритма FA
        self.population_fitness()  # Вычисляем фитнес для всей популяции
        for itr in range(self.Max):
            print("iteration = ", itr)
            self.a = self.a0 * pow(0.9, itr)  # Уменьшаем случайную интенсивность
            self.attract()  # Притяжение светлячков
            self.current_population()  # Отображение текущей популяции
            self.get_best()  # Получение лучшего решения
            gc.collect()  # Очистка памяти